## <center>Домашна работа</center>
# <center>Филтрирање на слика во фрекфенциски домен</center>

ТЕОРИЈА:

За да филтрираме слика во фреквенциски домен потребно е да ја помножиме сликата во фреквенциски домен со филтерот (кернел) кој е исто така во фреквенциски домен.$$I_{f,flit}=HI_f$$
За таа цел потребно е да се вметне Octave функција која ќе креира “Band pass gaussian filter” во фреквенциски домен. Bandpass гаусовиот филтер во фреквенциски домен е дефиниран со равенката:$$H(i,j)=e^{\frac{d(i,j)^2}{f_h^2}}$$
Каде што 𝑑(𝑖,𝑗) е евклидовото растојанието на даден пиксел од центарот на сликата, додека 𝑓ℎ и 𝑓 𝑙 се праговите на високите и ниските фреквенции кои сакаме да ги филтрираме. 
 
Конверзија на 2D сликата од просторен во фреквенциски домен се прави со користење на Фуриева трансформација. Фуриевата трансформација е дефинирана како: $$F(u,v)=\int\int_{-\infty}^\infty{𝑓(𝑥,𝑦)e^{−𝑖2𝜋(𝑢𝑥+𝑣𝑦)}dxdy}$$
А во дискретниот домен горната равенка може да биде преведена како$$F(u,v)={\sum_{m=-{\infty}}^{\infty}\sum_{n=-{\infty}}^{\infty}{f[m,n]∙e^{−𝑖2𝜋(𝑢𝑚𝑥_o+𝑣𝑛𝑦_o)}}}$$
Конверзија на 2D сликата од фреквенциски во просторен домен се прави со користење на инверзна Фуриева трансформација. Инверзната Фуриева трансформација е дефинирана како: $$F(x, y)=\int\int_{-\infty}^\infty{𝑓(u,v)e^{−𝑖2𝜋(𝑢𝑥+𝑣𝑦)}dudv}$$
А во дискретниот домен горната равенка може да биде преведена како$$F(x, y)={\sum_{m=-{\infty}}^{\infty}\sum_{n=-{\infty}}^{\infty}{f(m,n)∙e^{−𝑖2𝜋(xmu_o+ynv_o)}}}$$



## <center>Имплементација</center>

In [5]:
function filter = create_filter(nx,ny,d0,d1)
filter = ones(nx,ny);
for i = 0:nx-1
 for j = 0:ny-1
 dist= sqrt((i-nx/2)^2 + (j-ny/2)^2);
 filter(i+1,j+1) = exp(-(dist^2)/(d1^2)).*(1.0-exp(-(dist^2)/(d0^2)));
 end
end
filter(nx/2+1,ny/2+1)=1;
end

In [6]:
clear all
%read image
im = double(imread('Barbara.tif'));
%size of the image
[nx ny] = size(im);
%Transform the image from spatial to frequency domain
im_fft = fftshift(fft2(im));
% Create Gaussian bandpass kernels, in frequency domain, for 20 different cut off frequencies and filter the image
p=1;
for i = 1:10:101
q=1;
    for j = 1:10:101
        %Create the bandpass kernel
        tmp1 = create_filter(nx,ny,i,j);
        %Filter the image with the bandpass kernel and transform it back to spatial domain
        tmp = abs(ifft2(ifftshift(tmp1.*im_fft)));
        %Save the filtered image (in spatial domain) and kernel (in frequency domain) for visualization 
        filtered_image(p,q,:) = tmp(:);
        filter_kernel(p,q,:) = tmp1(:);
        q=q+1;
    end
    p=p+1;
end

In [7]:
im = double(imread('Barbara.tif'));
%size of the image
[nx ny] = size(im);
a = [nx ny];
save -6 file2.m a

In [8]:
save -6 file.m filtered_image
save -6 file1.m filter_kernel

In [9]:
from scipy.io import loadmat
import numpy as np

img = loadmat("file.m")
kernel = loadmat("file1.m")
niza = loadmat("file2.m")

trid = img['filtered_image'][1][5]
trid_kernel=kernel['filter_kernel'][1][5]

img_new = np.rot90(np.reshape(trid, (int(niza['a'][0][1]) , int(niza['a'][0][0]))))
img_new_kernel=np.rot90(np.reshape(trid_kernel, (int(niza['a'][0][1]) , int(niza['a'][0][0]))))


## <center>Визуелизација</center>

In [10]:
import plotly.graph_objects as go
import ipywidgets as widgets
from scipy.io import loadmat
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual

img = loadmat("file.m")
kernel = loadmat("file1.m")
niza = loadmat("file2.m")

def funkcija(change):
    trid = img['filtered_image'][izlez.value[0]][izlez.value[1]]
    trid_kernel=kernel['filter_kernel'][izlez.value[0]][izlez.value[1]]
    img_new = np.rot90(np.reshape(trid, (int(niza['a'][0][1]) , int(niza['a'][0][0]))))
    img_new_kernel=np.rot90(np.reshape(trid_kernel, (int(niza['a'][0][1]) , int(niza['a'][0][0]))))
    f1.data[0].z = img_new
    f2.data[0].z=img_new_kernel

izlez=widgets.IntRangeSlider(
    value=[0, 10],
    min=0,
    max=10,
    step=1,
    description='Cutoff:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

data=go.Heatmap(z=img_new,colorscale='gray',showscale=False)
data2=go.Heatmap(z=img_new_kernel,colorscale='gray',showscale=False)
figure = go.Figure(data=data)
figure2=go.Figure(data=data2)
f1 = go.FigureWidget(figure)
f2= go.FigureWidget(figure2)
izlez.observe(funkcija, names='value')
display(f1,f2,izlez)



FigureWidget({
    'data': [{'colorscale': [[0.0, 'rgb(0, 0, 0)'], [0.09090909090909091, 'rgb(16,
            …

FigureWidget({
    'data': [{'colorscale': [[0.0, 'rgb(0, 0, 0)'], [0.09090909090909091, 'rgb(16,
            …

IntRangeSlider(value=(0, 10), continuous_update=False, description='Cutoff:', max=10)